In [107]:
from flask import Flask, request, jsonify, render_template, session, redirect, url_for
from flask_session import Session
from openpyxl import load_workbook
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from flask import jsonify


app = Flask(__name__)
# Configure the Flask app for server-side session
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# Configure the Flask app for SQLAlchemy
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///bids.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)  # Keep this single initialization

class Bid(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), unique=True, nullable=False)
    customer_id = db.Column(db.Integer, db.ForeignKey('customer.id'))
    project_id = db.Column(db.Integer, db.ForeignKey('project.id'))
    # Explicitly specify the foreign_keys for clarity
    customer = db.relationship('Customer', backref='bid', lazy=True, foreign_keys=[customer_id])
    project = db.relationship('Project', backref='bid', lazy=True, foreign_keys=[project_id])
    category_budgets = db.relationship('CategoryBudget', backref='bid', lazy=True)
    labor_rates = db.relationship('LaborRate', backref='bid', lazy=True)


class Customer(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100))
    address = db.Column(db.String(200))
    state = db.Column(db.String(100))
    city = db.Column(db.String(100))
    zip_code = db.Column(db.String(20))
    # Bid relationship
    bid_id = db.Column(db.Integer, db.ForeignKey('bid.id'))

class Project(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100))
    address = db.Column(db.String(200))
    state = db.Column(db.String(100))
    city = db.Column(db.String(100))
    zip_code = db.Column(db.String(20))
    # Bid relationship
    bid_id = db.Column(db.Integer, db.ForeignKey('bid.id'))

class CategoryBudget(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(50))  # e.g., 'Drains', 'Irrigation', 'Landscape'
    amount = db.Column(db.Float)
    # Bid relationship
    bid_id = db.Column(db.Integer, db.ForeignKey('bid.id'))

class LaborRate(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    category = db.Column(db.String(50))  # e.g., 'Drains', 'Irrigation', 'Landscape'
    rate = db.Column(db.Float)
    # Bid relationship
    bid_id = db.Column(db.Integer, db.ForeignKey('bid.id'))


def load_sheets(filename):
    workbook = load_workbook(filename)
    inventory = []
    factors = []

    # Load Inventory Sheet
    inventory_sheet = workbook["Inventory"]  # Use the correct name of your Inventory sheet
    inventory_headers = [cell for cell in next(inventory_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in inventory_sheet.iter_rows(min_row=2, values_only=True):
        item = {inventory_headers[i]: value for i, value in enumerate(row)}
        inventory.append(item)
    
    # Load Factor Sheet
    factor_sheet = workbook["Factor"]  # Use the correct name of your Factor sheet
    factor_headers = [cell for cell in next(factor_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in factor_sheet.iter_rows(min_row=2, values_only=True):
        item = {factor_headers[i]: value for i, value in enumerate(row)}
        factors.append(item)
    
    return inventory, factors

inventory, factors = load_sheets('Input.xlsx')

@app.route('/get-bid/<int:bid_id>')
def get_bid(bid_id):
    print(f"Fetching bid with ID: {bid_id}")  # Print the ID of the bid being fetched
    bid = Bid.query.get(bid_id)
    if not bid:
        print("Bid not found")  # Print a message if the bid is not found
        return jsonify({"error": "Bid not found"}), 404

    # If bid is found, print out some basic details for debugging
    print(f"Found bid: {bid.name}, Customer ID: {bid.customer_id if bid.customer_id else 'N/A'}, Project ID: {bid.project_id if bid.project_id else 'N/A'}")

    bid_details = {
        "id": bid.id,
        "name": bid.name,
        "customer": {
            "id": bid.customer.id,
            "name": bid.customer.name,
            "address": bid.customer.address,
            "state": bid.customer.state,
            "city": bid.customer.city,
            "zip_code": bid.customer.zip_code
        } if bid.customer else {},
        "project": {
            "id": bid.project.id,
            "name": bid.project.name,
            "address": bid.project.address,
            "state": bid.project.state,
            "city": bid.project.city,
            "zip_code": bid.project.zip_code
        } if bid.project else {},
        "category_budgets": [{"category": cb.category, "amount": cb.amount} for cb in bid.category_budgets],
        "labor_rates": [{"category": lr.category, "rate": lr.rate} for lr in bid.labor_rates]
    }

    # Print the assembled details for the found bid
    print(f"Returning details for bid: {bid_details}")
    
    return jsonify(bid_details)


@app.route('/new-bid')
def new_bid():
    session.setdefault('bid_items', [])
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return render_template('Bid_Job_Estimating.html', bid_items=session['bid_items'], total_cost=total_cost)

@app.route('/save-bid', methods=['POST'])
def save_bid():
    data = request.get_json()
    print("Received data:", data)  # Debugging purpose

    try:
        if 'id' in data and data['id']:
            bid = Bid.query.get(data['id'])
            if not bid:
                return jsonify({"error": "Bid not found"}), 404
            print("Updating existing bid:", bid.id)
        else:
            bid = Bid()
            print("Creating new bid")

        bid.name = data.get('name')
        print("Bid name set to:", bid.name)

        # Handling Customer
        customer_data = data.get('customer', {})
        if customer_data:
            customer = Customer.query.filter_by(name=customer_data.get('name')).first()
            if not customer:
                customer = Customer()
            customer.name = customer_data.get('name')
            customer.address = customer_data.get('address')
            customer.state = customer_data.get('state')
            customer.city = customer_data.get('city')
            customer.zip_code = customer_data.get('zip_code')
            db.session.merge(customer)  # Merge handles both insert and update
            bid.customer = customer

        # Handling Project
        project_data = data.get('project', {})
        if project_data:
            project = Project.query.filter_by(name=project_data.get('name')).first()
            if not project:
                project = Project()
            project.name = project_data.get('name')
            project.address = project_data.get('address')
            project.state = project_data.get('state')
            project.city = project_data.get('city')
            project.zip_code = project_data.get('zip_code')
            db.session.merge(project)  # Merge handles both insert and update
            bid.project = project

        if 'date' in data:
            bid.date = datetime.strptime(data['date'], '%Y-%m-%d')

        db.session.add(bid)
        db.session.commit()
        print("Bid saved successfully with ID:", bid.id)
        return jsonify({"message": "Bid saved successfully", "bid_id": bid.id}), 200
    except IntegrityError as e:
        db.session.rollback()
        return jsonify({"error": "A bid with the given name already exists."}), 400


@app.route('/')
def home():
    # Basic homepage with links to other parts of the application
    return render_template('Home.html')

@app.route('/add', methods=['POST'])
def add_item():
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        # Append to bid_items in session
        session['bid_items'].append({**item, "Quantity": quantity, "Cost": float(item['Cost'])})
        session.modified = True  # Mark the session as modified to save changes
        total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
        return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    else:
        return jsonify(success=False)

@app.route('/delete', methods=['POST'])
def delete_items():
    selected_items = request.json['selectedItems']
    # Assuming 'bid_items' is stored in session and is a list of dictionaries
    session['bid_items'] = [item for item in session.get('bid_items', []) if item['PartNum'] not in selected_items]
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return jsonify({'bid_items': session['bid_items'], 'total_cost': total_cost})

@app.route('/inventory')
def get_inventory():
    return jsonify(inventory)

@app.route('/factors')
def get_factors():
    # Assuming factors is a list of dictionaries
    try:
        # Debug print to inspect the structure
        print(factors)
        return jsonify(factors)
    except TypeError as e:
        # Log the error for debugging
        print(f"Error serializing factors: {e}")
        # Return an error message or empty list as a fallback
        return jsonify([]), 500

@app.route('/add-update-bid', methods=['GET', 'POST'])
def add_update_bid():
    if request.method == 'POST':
        bid_id = request.form.get('bidId')  # Get the Bid ID from the form
        bid_name = request.form.get('BidName')

        if bid_id:
            # Attempt to update an existing bid
            bid = Bid.query.get(bid_id)
            if bid:
                bid.name = bid_name
                # Update other fields as necessary
                db.session.commit()
                print(f"Updated existing bid with ID: {bid_id}")  # Debugging purpose
                return redirect(url_for('new_bid', bid_id=bid.id))  # Redirect to a page showing the updated bid
            else:
                return jsonify({"error": "Bid not found"}), 404
        else:
            # Create a new bid
            existing_bid = Bid.query.filter_by(name=bid_name).first()
            if existing_bid is None:
                new_bid = Bid(name=bid_name)
                # Set other attributes for the new bid as necessary
                db.session.add(new_bid)
                db.session.commit()
                print(f"Created new bid with name: {bid_name}")  # Debugging purpose
                return redirect(url_for('new_bid', bid_id=new_bid.id))  # Redirect to a page showing the new bid
            else:
                # A bid with this name already exists
                return jsonify({"error": "A bid with this name already exists."}), 400
    else:
        # For a GET request, show the form to add or update a bid
        bid_id = request.args.get('bid_id')  # Assuming you pass the bid ID as a query parameter for updates
        bid_details = None
        if bid_id:
            bid = Bid.query.get(bid_id)
            if bid:
                bid_details = {
                    "id": bid.id,
                    "name": bid.name,
                    # Include other necessary bid details here
                }
            else:
                return jsonify({"error": "Bid not found"}), 404
        return render_template('AddUpdateBid.html', bid_details=bid_details)


@app.route('/check-bid', methods=['POST'])
def check_bid():
    data = request.get_json()
    bid_name = data.get('bidName')
    bid = Bid.query.filter_by(name=bid_name).first()
    
    if bid:
        print(f"Found bid: {bid.name}")  # Print bid name

        # Fetching related customer and project details
        customer_details = {}
        if bid.customer:
            customer_details = {
                "name": bid.customer.name,
                "address": bid.customer.address,
                "state": bid.customer.state,
                "city": bid.customer.city,
                "zip_code": bid.customer.zip_code
            }
            print(f"Customer details: {customer_details}")  # Print customer details

        project_details = {}
        if bid.project:
            project_details = {
                "name": bid.project.name,
                "address": bid.project.address,
                "state": bid.project.state,
                "city": bid.project.city,
                "zip_code": bid.project.zip_code
            }
            print(f"Project details: {project_details}")  # Print project details

        # Fetching category budgets and labor rates
        category_budgets = [{
            "category": cb.category,
            "amount": cb.amount
        } for cb in bid.category_budgets]
        print(f"Category budgets: {category_budgets}")  # Print category budgets

        labor_rates = [{
            "category": lr.category,
            "rate": lr.rate
        } for lr in bid.labor_rates]
        print(f"Labor rates: {labor_rates}")  # Print labor rates

        bid_details = {
            "exists": True,
            "details": {
                "name": bid.name,
                "customer": customer_details,
                "project": project_details,
                "category_budgets": category_budgets,
                "labor_rates": labor_rates,
                # Add other relevant details here
            }
        }
    else:
        bid_details = {"exists": False}
        print("Bid not found")  # Print message if bid is not found
    
    return jsonify(bid_details)


@app.route('/create-bid', methods=['POST'])
def create_bid():
    data = request.get_json()  # Assuming you're sending data as JSON
    bid_name = data.get('bidName')  # Make sure this matches the name in your JavaScript

    if bid_name:
        try:
            new_bid = Bid(name=bid_name)
            # Set other necessary fields of the Bid object here
            db.session.add(new_bid)
            db.session.commit()
            return jsonify({"message": "Bid created successfully", "bid_id": new_bid.id}), 200
        except Exception as e:
            db.session.rollback()
            return jsonify({"error": str(e)}), 500
    else:
        return jsonify({"error": "Bid name is required"}), 400


@app.route('/update-bid', methods=['POST'])
def update_bid():
    bid_name = request.form.get('bidName')
    # Find the existing bid and update it
    bid = Bid.query.filter_by(name=bid_name).first()
    if bid:
        # Update bid details
        db.session.commit()
    # Redirect or return a response
    return redirect(url_for('bid_job_estimating'))

@app.route('/bid-job-estimating')
def bid_job_estimating():
    # Render your Bid_Job_Estimating.html template or handle the logic
    return render_template('Bid_Job_Estimating.html')

@app.route('/insert', methods=['POST'])
def insert_item():
    insert_after = request.form['InsertAfter']
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        index = next((i for i, item in enumerate(session['bid_items']) if item['PartNum'] == insert_after), None)
        if index is not None:
            session['bid_items'].insert(index + 1, {**item, "Quantity": quantity, "Cost": float(item['Cost'])})
            session.modified = True  # Mark the session as modified to save changes
            total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
            return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    return jsonify(success=False)

if __name__ == '__main__':
    with app.app_context():
        db.create_all()  # This should now work without throwing an error
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Feb/2024 08:29:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 08:29:54] "GET /new-bid HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 08:29:54] "GET /inventory HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 08:29:54] "GET /factors HTTP/1.1" 200 -


[{'Factor_ID': '100', 'Description': '1/2 LATERAL FIT 45%', 'Labor Hours': 0.0408}, {'Factor_ID': '1000', 'Description': '2" GALV FIT', 'Labor Hours': 0.05}, {'Factor_ID': '1000.1', 'Description': '2.5" GALV FIT', 'Labor Hours': 0.07}, {'Factor_ID': '1000.2', 'Description': '2" GALV FIT,VIT', 'Labor Hours': 0.05}, {'Factor_ID': '1001', 'Description': '1 1/2" GALV FIT', 'Labor Hours': 0.045}, {'Factor_ID': '1001.1', 'Description': '1 1/2" GALV FIT,VIT', 'Labor Hours': 0.042}, {'Factor_ID': '1002', 'Description': '1 1/4" GALV FITTINGS', 'Labor Hours': 0.033}, {'Factor_ID': '1002.1', 'Description': '1 1/4" GALV FIT,VIT', 'Labor Hours': 0.03}, {'Factor_ID': '1003', 'Description': '1" GALV. FIT.', 'Labor Hours': 0.03}, {'Factor_ID': '1003.1', 'Description': '1" GALV FIT,VIT', 'Labor Hours': 0.027}, {'Factor_ID': '1004', 'Description': '3/4" GALV. FIT.', 'Labor Hours': 0.0285}, {'Factor_ID': '1004.1', 'Description': '3/4"  GALV FIT,VIT', 'Labor Hours': 0.023}, {'Factor_ID': '1005', 'Descript

127.0.0.1 - - [29/Feb/2024 08:29:56] "GET /add-update-bid HTTP/1.1" 200 -
127.0.0.1 - - [29/Feb/2024 08:29:59] "POST /check-bid HTTP/1.1" 200 -


Bid not found


127.0.0.1 - - [29/Feb/2024 08:30:00] "POST /create-bid HTTP/1.1" 400 -
